In [ ]:
# Core PyTorch and vision libraries (T4 supports CUDA 12.1)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --quiet

# Transformers, Hugging Face Hub, and model optimizations
!pip install -U "transformers>=4.47.0" "huggingface_hub>=0.23" accelerate bitsandbytes pillow einops --quiet

# For image loading and processing
!pip install -U sentence-transformers --quiet

# Optional: For quantization (if you want to save memory)
!pip install bitsandbytes --quiet
!pip install pymongo pillow --quiet



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 92.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.49.1 requires pillow<12.0,>=8.0, but you have pillow 12.0.0 which is incompatible.


In [ ]:
!pip install --force-reinstall --no-cache-dir pillow==10.3.0 torchvision==0.18.0 --quiet
!pip install --force-reinstall --no-cache-dir transformers --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 144.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 138.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 137.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 204.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 119.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 141.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image


In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
# MongoDB connection
mongo_uri = "your_mongodb_connection_string"  # Replace with your MongoDB URI
client = MongoClient(mongo_uri)
db = client["your_database_name"]  # Replace with your database name
collection = db["wardrobe_items"]  # Replace with your collection name

In [ ]:
model_id = "HuggingFaceTB/SmolVLM-Instruct"

from transformers import AutoProcessor, AutoModelForImageTextToText
import torch

processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    dtype=torch.bfloat16,  # Use 'dtype' instead of 'torch_dtype'
    device_map="auto"      # Automatically handles device placement
)


generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [ ]:
def load_image_from_db(image_id):
    """
    Load image from MongoDB by ID
    Assumes images are stored as base64 strings or binary data
    """
    doc = collection.find_one({"_id": image_id})

    if doc is None:
        raise ValueError(f"Image with ID {image_id} not found")

    # If image is stored as base64 string
    if "image_data" in doc and isinstance(doc["image_data"], str):
        image_bytes = base64.b64decode(doc["image_data"])
        image = Image.open(io.BytesIO(image_bytes))
        return image

    # If image is stored as binary data
    elif "image_data" in doc:
        image = Image.open(io.BytesIO(doc["image_data"]))
        return image

    # If image path/URL is stored, and you have local files
    elif "image_path" in doc:
        image = Image.open(doc["image_path"])
        return image

    else:
        raise ValueError("No valid image data found in document")

def get_user_wardrobe_images(user_id, limit=None):
    """
    Get all images for a user's wardrobe
    """
    query = {"user_id": user_id}
    cursor = collection.find(query)

    if limit:
        cursor = cursor.limit(limit)

    images = []
    for doc in cursor:
        try:
            image = load_image_from_db(doc["_id"])
            images.append({
                "image": image,
                "metadata": {
                    "id": str(doc["_id"]),
                    "type": doc.get("type", "unknown"),
                    "color": doc.get("color", "unknown"),
                    "season": doc.get("season", "all")
                }
            })
        except Exception as e:
            print(f"Error loading image {doc['_id']}: {e}")

    return images


In [ ]:
def recommend_outfits(user_id, occasion, location, weather_info):
    """
    Recommend outfits based on user's wardrobe, occasion, and weather
    """
    # Get user's wardrobe images
    wardrobe_items = get_user_wardrobe_images(user_id)

    if not wardrobe_items:
        return "No wardrobe items found for this user"

    # Extract just the images
    images = [item["image"] for item in wardrobe_items]

    # Create prompt with context
    prompt_text = f"""You are a fashion AI assistant. Based on the user's wardrobe images shown,
    recommend 5 suitable outfit combinations for the following:

    Occasion: {occasion}
    Location: {location}
    Weather: {weather_info}

    For each outfit, specify which items from the wardrobe to combine and why they work well together."""

    # Prepare messages for the model
    messages = [
        {
            "role": "user",
            "content": [
                *[{"type": "image"} for _ in images],  # Add image placeholders
                {"type": "text", "text": prompt_text}
            ]
        }
    ]

    # Process and generate
    prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(text=prompt, images=images, return_tensors="pt")
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}

    # Generate response
    generated_ids = model.generate(**inputs, max_new_tokens=1000)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return generated_text


In [ ]:
#delete this
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)
print(generated_texts[0])


KeyboardInterrupt: 